In [39]:
from tensorflow.keras.layers import Flatten, Dense
from tensorflow.keras import models
from keras.datasets import mnist
from tensorflow.keras.utils import to_categorical

In [35]:
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

In [40]:
model = models.Sequential([Flatten(input_shape=(28, 28)), Dense(128, activation='relu'),
    Dense(64, activation='relu'), Dense(10, activation='softmax')])

In [41]:
train_images = train_images.reshape((60000, 28, 28, 1))
train_images = train_images.astype('float32') / 255
test_images = test_images.reshape((10000, 28, 28, 1))
test_images = test_images.astype('float32') / 255
train_labels = to_categorical(train_labels)
test_labels = to_categorical(test_labels)


model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(train_images, train_labels, epochs=5, batch_size=64)
test_loss, test_acc = model.evaluate(test_images, test_labels)

print(test_acc)
model.save('Recoginizer.h5')

Epoch 1/5
938/938 [==============================] - 6s 5ms/step - loss: 0.2670 - accuracy: 0.9226
Epoch 2/5
938/938 [==============================] - 5s 5ms/step - loss: 0.1151 - accuracy: 0.9653
Epoch 3/5
938/938 [==============================] - 5s 5ms/step - loss: 0.0801 - accuracy: 0.9763
Epoch 4/5
938/938 [==============================] - 5s 5ms/step - loss: 0.0632 - accuracy: 0.9804
Epoch 5/5
313/313 [==============================] - 1s 3ms/step - loss: 0.0795 - accuracy: 0.9776
0.9775999784469604


In [42]:
from tkinter import *
import cv2
import numpy as np
from PIL import ImageGrab
from tensorflow.keras.models import load_model

model = load_model('Recoginizer.h5')

In [ ]:
root = Tk()
lastx, lasty = None, None

cv = Canvas(root, width=300, height=300, bg='white')
cv.grid(row=0, column=0, pady=2, sticky=W, columnspan=2)


def draw_lines(event):
    global lastx, lasty
    x, y = event.x, event.y
    cv.create_line((lastx, lasty, x, y), width=8, fill='black', capstyle=ROUND, smooth=TRUE, splinesteps=12)
    lastx, lasty = x, y


def activate_event(event):
    global lastx, lasty
    cv.bind('<B1-Motion>', draw_lines)
    lastx, lasty = event.x, event.y


cv.bind('<Button-1>', activate_event)


def Recognize_Digit():
    filename = f'Image.png'
    widget = cv

    x = root.winfo_rootx() + widget.winfo_rootx()
    y = root.winfo_rooty() + widget.winfo_rooty()
    x1 = x + widget.winfo_width()
    y1 = y + widget.winfo_height()
    ImageGrab.grab().crop((x, y, x1, y1)).save(filename)

    image = cv2.imread(filename, cv2.IMREAD_COLOR)
    gray = cv2.cvtColor(image.copy(), cv2.COLOR_BGR2GRAY)
    ret, th = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)
    contour = cv2.findContours(th, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)[0]
    x, y, w, h = cv2.boundingRect(contour[0])
    cv2.rectangle(image, (x, y), (x + w, y + h), (255, 0, 0), 1)
    digit = th[y:y + h, x:x + w]

    resized_digit = cv2.resize(digit, (18, 18))
    padded_digit = np.pad(resized_digit, ((5, 5), (5, 5)), "constant", constant_values=0)
    digit = (padded_digit.reshape(1, 28, 28, 1)) / 255.0
    pred = model.predict([digit])[0] 
    print(str(np.argmax(pred)) + ' ' + str(int(max(pred) * 100)) + '%')

btn_save = Button(text='Recognize Digit', command=Recognize_Digit)
btn_save.grid(row=2, column=0, pady=1, padx=1,columnspan=2)
root.mainloop()

8 99%
